In [ ]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.applications import MobileNetV2

#Obtener dataset

In [ ]:
# Cargar el conjunto de datos CIFAR-10
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# Normalizar las imágenes
x_train = x_train / 255.0
x_test = x_test / 255.0

# Convertir las etiquetas a one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

170498071/170498071 [==============================] - 6s 0us/step


In [ ]:
# Crear el modelo
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(10, activation='softmax'))



In [ ]:
# Compilar el modelo
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(lr=0.001),
              metrics=['accuracy'])

In [ ]:
# Entrenar el modelo
batch_size = 128
epochs = 50

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,
          validation_data=(x_test, y_test), shuffle=True)

Epoch 1/50
391/391 [==============================] - 209s 531ms/step - loss: 1.5733 - accuracy: 0.4271 - val_loss: 1.1957 - val_accuracy: 0.5756
Epoch 2/50
391/391 [==============================] - 203s 519ms/step - loss: 1.1247 - accuracy: 0.5994 - val_loss: 0.9395 - val_accuracy: 0.6710
Epoch 3/50
391/391 [==============================] - 204s 521ms/step - loss: 0.9600 - accuracy: 0.6598 - val_loss: 0.8597 - val_accuracy: 0.6986
Epoch 4/50
391/391 [==============================] - 204s 521ms/step - loss: 0.8565 - accuracy: 0.6993 - val_loss: 0.8077 - val_accuracy: 0.7229
Epoch 5/50
391/391 [==============================] - 203s 518ms/step - loss: 0.7836 - accuracy: 0.7255 - val_loss: 0.7195 - val_accuracy: 0.7474
Epoch 6/50
391/391 [==============================] - 204s 521ms/step - loss: 0.7254 - accuracy: 0.7437 - val_loss: 0.7302 - val_accuracy: 0.7454
Epoch 7/50
391/391 [==============================] - 205s 526ms/step - loss: 0.6732 - accuracy: 0.7645 - val_loss: 0.6953 -

In [ ]:
# Evaluar el modelo en el conjunto de pruebas
score_model = model.evaluate(x_test, y_test, verbose=0)
print('Test accuracy:', score_model[1])


#Base Model

In [ ]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

In [ ]:
# Congelar las capas del modelo base para que no se actualicen durante el entrenamiento
for layer in base_model.layers:
    layer.trainable = False

#Transfer learning

In [ ]:
# Crear el modelo final utilizando el modelo base y agregar capas densas
model = Sequential()
model.add(Flatten(input_shape=base_model.output_shape[1:]))
model.add(Dense(10, activation='softmax'))

# Combinar el modelo base con el modelo final
transfer_model = Sequential()
transfer_model.add(base_model)
transfer_model.add(model)

# Compilar el modelo
transfer_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
transfer_model.fit(x_train, y_train, epochs=10, batch_size=128, validation_data=(x_test, y_test))

Epoch 1/10
391/391 [==============================] - 9s 17ms/step - loss: 2.1398 - accuracy: 0.2547 - val_loss: 2.0416 - val_accuracy: 0.2839
Epoch 2/10
391/391 [==============================] - 5s 12ms/step - loss: 1.9976 - accuracy: 0.2970 - val_loss: 1.9732 - val_accuracy: 0.2971
Epoch 3/10
391/391 [==============================] - 6s 14ms/step - loss: 1.9490 - accuracy: 0.3073 - val_loss: 1.9435 - val_accuracy: 0.3029
Epoch 4/10
391/391 [==============================] - 5s 12ms/step - loss: 1.9246 - accuracy: 0.3120 - val_loss: 1.9275 - val_accuracy: 0.3072
Epoch 5/10
391/391 [==============================] - 5s 13ms/step - loss: 1.9099 - accuracy: 0.3152 - val_loss: 1.9175 - val_accuracy: 0.3089
Epoch 6/10
391/391 [==============================] - 5s 14ms/step - loss: 1.9002 - accuracy: 0.3177 - val_loss: 1.9106 - val_accuracy: 0.3092
Epoch 7/10
391/391 [==============================] - 5s 13ms/step - loss: 1.8931 - accuracy: 0.3187 - val_loss: 1.9062 - val_accuracy: 0.3116

In [ ]:
# Evaluar el modelo en el conjunto de pruebas
score_transfer = transfer_model.evaluate(x_test, y_test, verbose=0)
print('Test accuracy:', score_transfer[1])

Test accuracy: 0.3156000077724457


#Fine Tuning

In [ ]:
# Descongelar las ultimas 10 capas del modelo base para que se actualicen durante el entrenamiento
for layer in base_model.layers[-10:]:
    layer.trainable = True

In [ ]:
# Crear el modelo final utilizando el modelo base y agregar capas densas
model_tuning = Sequential()


model_tuning.add(Conv2D(1280, (5, 5), activation='relu', padding='same'))
model_tuning.add(Conv2D(1280, (5, 5), activation='relu', padding='same'))

model_tuning.add(Flatten(input_shape=base_model.output_shape[1:]))

model_tuning.add(Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.25))
model_tuning.add(Dense(256, activation='relu'))
model_tuning.add(Dense(256, activation='relu'))

model_tuning.add(Dense(10, activation='softmax'))

# Combinar el modelo base con el modelo final
final_model = Sequential()
final_model.add(base_model)
final_model.add(model_tuning)

# Compilar el modelo
final_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Entrenar el modelo
final_model.fit(x_train, y_train, epochs=10, batch_size=128, validation_data=(x_test, y_test))

Epoch 1/10
391/391 [==============================] - 96s 229ms/step - loss: 1.4785 - accuracy: 0.4829 - val_loss: 2.5325 - val_accuracy: 0.2119
Epoch 2/10
391/391 [==============================] - 90s 229ms/step - loss: 1.2480 - accuracy: 0.5697 - val_loss: 1.9279 - val_accuracy: 0.3282
Epoch 3/10
391/391 [==============================] - 90s 231ms/step - loss: 1.1552 - accuracy: 0.6005 - val_loss: 2.4417 - val_accuracy: 0.2318
Epoch 4/10
391/391 [==============================] - 91s 233ms/step - loss: 1.0718 - accuracy: 0.6257 - val_loss: 1.9739 - val_accuracy: 0.3331
Epoch 5/10
391/391 [==============================] - 91s 233ms/step - loss: 1.0023 - accuracy: 0.6485 - val_loss: 3.7223 - val_accuracy: 0.2423
Epoch 6/10
391/391 [==============================] - 91s 234ms/step - loss: 0.9295 - accuracy: 0.6704 - val_loss: 3.5296 - val_accuracy: 0.2332
Epoch 7/10
391/391 [==============================] - 92s 235ms/step - loss: 0.8535 - accuracy: 0.6962 - val_loss: 2.4354 - val_ac

In [ ]:
# Evaluar el modelo en el conjunto de pruebas
score_tuning = final_model.evaluate(x_test, y_test, verbose=0)
print('Test accuracy:', score_tuning[1])

Test accuracy: 0.4438999891281128
